In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
df=pd.read_csv('Student_Performance.csv')
print(df.shape)
print(df.columns)

(10000, 6)
Index(['Hours Studied', 'Previous Scores', 'Extracurricular Activities',
       'Sleep Hours', 'Sample Question Papers Practiced', 'Performance Index'],
      dtype='object')


In [3]:
df.isnull().sum() #no null values

Hours Studied                       0
Previous Scores                     0
Extracurricular Activities          0
Sleep Hours                         0
Sample Question Papers Practiced    0
Performance Index                   0
dtype: int64

In [4]:
df['Extracurricular Activities'] = df['Extracurricular Activities'].map({'Yes': 1, 'No': 0})
#label encoding for categorical variable

In [5]:
#IQR for outliers
Q1 = df.quantile(0.25, numeric_only=True)
Q3 = df.quantile(0.75, numeric_only=True)
IQR = Q3 - Q1

# Detect outliers
outliers = ((df[Q1.index] < (Q1 - 1.5 * IQR)) | (df[Q1.index] > (Q3 + 1.5 * IQR)))

# Optional: View rows with any outlier
df[outliers.any(axis=1)]

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index


In [6]:
# Define your target column
target_col = 'Performance Index'

# Get the correlation matrix
corr_matrix = df.corr(numeric_only=True)

# Get absolute correlation of all features with the target
target_corr = corr_matrix[target_col].abs()

# Sort correlations in descending order
sorted_corr = target_corr.sort_values(ascending=False)

# Set a correlation threshold (e.g., 0.3)
threshold = 0.3
selected_features = sorted_corr[sorted_corr > threshold].index.tolist()

# Remove target from the selected features list if needed
selected_features = [col for col in selected_features if col != target_col]

print("✅ Selected Features Based on Correlation:")
print(selected_features)

# Optionally, create a new DataFrame with selected features
df_selected = df[selected_features + [target_col]]


✅ Selected Features Based on Correlation:
['Previous Scores', 'Hours Studied']


In [11]:
#scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numerical_cols = ['Hours Studied', 'Previous Scores']
df_selected[numerical_cols] = scaler.fit_transform(df_selected[numerical_cols])

C:\Users\NikitaRampurkarV-Sof\AppData\Local\Temp\ipykernel_2336\4022462724.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected[numerical_cols] = scaler.fit_transform(df_selected[numerical_cols])


In [12]:
from sklearn.model_selection import train_test_split

X = df_selected.drop('Performance Index', axis=1)
y = df_selected['Performance Index']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
df_selected.to_csv("preprocessed_student2.csv", index=False)